In [1]:
import pandas as pd
import zipfile
import xlrd
import re
import os

In [2]:
x=zipfile.ZipFile(f'..//Final_Project/Altadis/FY1415/Master_Cigarrillos_FY1415(Septiembre).zip')
df=pd.read_excel(x.open('Master_Cigarrillos_FY1415(Septiembre).xlsx'))
keep=[i for i in df.columns[0:28]]

In [9]:
lista=[]
data_cigarrillos=pd.DataFrame()
def concat_files(x):
    for dirpath, dirnames, filenames in os.walk(x):
        for filename in filenames:
            for x in filename:
                for i in range(10,20):
                    if filename.endswith(f"Master_Cigarrillos_FY{i}{i+1}(Septiembre).zip"):
                        path=os.path.join(dirpath, filename)
                        zipp=zipfile.ZipFile(path)
                        print(os.path.exists(f'./Master_Cigarrillos_FY{i}{i+1}(Septiembre).xlsx'))
                        if os.path.exists(f'./Master_Cigarrillos_FY{i}{i+1}(Septiembre).xlsx')==True:
                            data=pd.read_excel(zipp.open(f"Master_Cigarrillos_FY{i}{i+1}(Septiembre).xlsx"))
                            data = data.loc[:, df.columns.isin(keep)]
                            data['Año']=f'FY{i}{i+1}'
                        else:
                            data=pd.read_excel(zipp.open(f"Master_Cigarrillos_FY{i}{i+1}(Septiembre).xls"))
                            data = data.loc[:, df.columns.isin(keep)]
                            data['Año']=f'FY{i}{i+1}'
                            pass
    lista.append(data)
    data_cigarrillos=pd.concat(data_cigarrillos,data)
    return  data_cigarrillos

In [10]:
print(concat_files('/home/carmencuadrado/Ironhack/Final_Project/Altadis'))

False


KeyError: "There is no item named 'Master_Cigarrillos_FY1415(Septiembre).xls' in the archive"

In [ ]:
data_picadura=pd.DataFrame()
def concat_files_picadura(x):
    for dirpath, dirnames, filenames in os.walk(x):
        for filename in filenames:
            for x in filename:
                for i in range(10,20):
                    if filename.endswith(f"Master_Picadura_FY{i}{i+1}(Septiembre).zip"):
                        path=os.path.join(dirpath, filename)
                        zipp=zipfile.ZipFile(path)
                        if os.path.exists(f'./Master_Picadura_FY{i}{i+1}(Septiembre).xls')==True:
                            data=pd.read_excel(zipp.open(f"Master_Picadura_FY{i}{i+1}(Septiembre).xls"))
                            data = data.loc[:, df.columns.isin(keep)]
                            data['Año']=f'FY{i}{i+1}'
                        else:
                            data=pd.read_excel(zipp.open(f"Master_Picadura_FY{i}{i+1}(Septiembre).xlsx"))
                            data = data.loc[:, df.columns.isin(keep)]
                            data['Año']=f'FY{i}{i+1}'
    lista.append(data)
    data_picadura=pd.concat(data_picadura,data)
    return  data_picadura 

In [ ]:
data_cigarros=pd.DataFrame()
def concat_files_picadura(x):
    for dirpath, dirnames, filenames in os.walk(x):
        for filename in filenames:
            for x in filename:
                for i in range(10,20):
                    if filename.endswith(f"Master_Cigarros_FY{i}{i+1}(Septiembre).zip"):
                        path=os.path.join(dirpath, filename)
                        zipp=zipfile.ZipFile(path)
                        if os.path.exists(f'./Master_Cigarros_FY{i}{i+1}(Septiembre).xls')==True:
                            data=pd.read_excel(zipp.open(f"Master_Cigarros_FY{i}{i+1}(Septiembre).xls"))
                            data = data.loc[:, df.columns.isin(keep)]
                            data['Año']=f'FY{i}{i+1}'
                        else:
                            data=pd.read_excel(zipp.open(f"Master_Cigarros_FY{i}{i+1}(Septiembre).xlsx"))
                            data = data.loc[:, df.columns.isin(keep)]
                            data['Año']=f'FY{i}{i+1}'
    lista.append(data)
    data_cigarros=pd.concat(data_cigarros,data)
    return  data_cigarros

In [ ]:
data_cigarrillos

In [ ]:
data_picadura

In [ ]:
data_cigarros